<a href="https://colab.research.google.com/github/Gustavo-2212/Automacao-de-Envio-de-Relatorios/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

bd = pd.read_excel(r'/content/drive/MyDrive/Aula 01/Vendas.xlsx')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
display(bd)

In [6]:
# Calculando o faturamento por loja
faturamento = bd[['ID Loja', 'Valor Final']].groupby('ID Loja').sum()

# Ordenando os valores
faturamento = faturamento.sort_values(by='Valor Final', ascending=False)

In [7]:
# Calculando e organizando os dados da quantidade vendida por loja
qntLoja = bd[['ID Loja', 'Quantidade']].groupby('ID Loja').sum()
qntLoja = qntLoja.sort_values(by='ID Loja', ascending=False)

In [8]:
# Calculo do Ticket Médio por Loja TM = Faturamento(Loja) / Quantidade(Loja)
TM = (faturamento['Valor Final'] / qntLoja['Quantidade']).to_frame()
TM = TM.rename(columns = {0: 'Ticket Medio'})
TM = TM.sort_values(by='Ticket Medio', ascending=False)

In [9]:
# Função para envio de email com o relatório
import smtplib
import email.message

def enviarEmail(relatorio, loja, senha, emailDestino, emailEmissor):
  
  server = smtplib.SMTP('smtp.gmail.com:587')
  email_content = f'''
    <p>Prezado, segue o relatório da loja {loja}<p>
    {relatorio.to_html()}
    <p>Obrigado<p>
  '''

  msg = email.message.Message()
  msg['Subject'] = f'Relatório Loja: {loja}'

  msg['From'] = emailEmissor
  msg['To'] = emailDestino

  # O ideal é que você crie area para que o dono do email remetente informe sua senha e email
  #senha = '...'
  #password = senha
  msg.add_header('Content-Type', 'text/html')
  msg.set_payload(email_content)

  s = smtplib.SMTP('smtp.gmail.com: 587')
  s.starttls()

  s.login(msg['From'], senha)
  s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))


In [ ]:
import getpass

lojas = bd['ID Loja'].unique()

emailEmissor = str(input('Digite seu email: '))
emailDestino = str(input('Digite o email alvo: '))
senha = getpass.getpass(prompt='Digite a senha de seu email: ', stream=None)

for loja in lojas:
  tabela_loja = bd.loc[bd['ID Loja'] == loja, ['ID Loja', 'Quantidade', 'Valor Final']]
  resumo_loja = tabela_loja.groupby( 'ID Loja').sum()
  resumo_loja['Ticket Medio'] = resumo_loja['Valor Final'] / resumo_loja['Quantidade']

  enviarEmail(resumo_loja, loja, senha, emailDestino, emailEmissor)

tabela_diretoria = faturamento.join(qntLoja).join(TM)
enviarEmail(tabela_diretoria, 'Todas as Lojas', senha, emailDestino, emailEmissor)